In [5]:
import pandas as pd
from IPython.display import Markdown
import re

df = pd.read_csv("../results/trace_resolved.csv")
df.head()

,timestamp_ns,thread_id,event,callee_addr,callee_func,callee_fileline,caller_addr,caller_func,caller_fileline
0,1751729557632068189,51361,ENTER,0x402909,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...,0x40901d,__libc_csu_init,??:?
1,1751729557632126794,51361,ENTER,0x40286a,"__static_initialization_and_destruction_0(int,...",/home/nhorro/workspace/learn-code-analysis/01_...,0x402933,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...
2,1751729557632263852,51361,EXIT,0x40286a,"__static_initialization_and_destruction_0(int,...",/home/nhorro/workspace/learn-code-analysis/01_...,0x402933,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...
3,1751729557632270458,51361,EXIT,0x402909,_GLOBAL__sub_I_main,/home/nhorro/workspace/learn-code-analysis/01_...,0x40901d,__libc_csu_init,??:?
4,1751729557632275265,51361,ENTER,0x40260b,main,/home/nhorro/workspace/learn-code-analysis/01_...,0x7ff21a89a083,??,??:0


In [6]:
import re

def is_relevant(func):
    if func.startswith("main"): return True
    if func.startswith("Manager::"): return True
    if func.startswith("Worker::"): return True
    if func.startswith("Reporter::"): return True
    return False

df_logic = df[df['callee_func'].apply(is_relevant)]
df_logic.callee_func.unique()

array(['main', 'Manager::Manager()', 'Worker::Worker()',
       'Worker::onJobDone(std::function<void (CalcJob const&)>)',
       'Manager::Manager()::{lambda(CalcJob const&)#1} const& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>() const',
       'Manager::Manager()::{lambda(CalcJob const&)#1}& std::_Any_data::_M_access<Manager::Manager()::{lambda(CalcJob const&)#1}>()',
       'Manager::start()', 'Worker::start()',
       'Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)',
       'Worker::start()::{lambda()#1}& std::__get_helper<0ul, Worker::start()::{lambda()#1}>(std::_Tuple_impl<0ul, Worker::start()::{lambda()#1}>&)',
       'Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}&&>(std::remove_reference<Worker::start()::{lambda()#1}&&>::type&)',
       'Worker::start()::{lambda()#1}::operator()() const',
       'Manager::submitJob(CalcJob const&)'

In [13]:
import re

# 1️⃣ Define función de normalización de nombre legible
def pretty_func(name):
    if not isinstance(name, str):
        return "Unknown"
    # Simplifica lambdas de Manager
    if "Manager::Manager()::{lambda" in name:
        return "Manager::onJobDone()"
    if "Worker::run()::{lambda" in name:
        return "Worker::waitCondition()"
    # Otras transformaciones específicas aquí si quieres...
    return name

# 2️⃣ Define patrón de funciones relevantes
def is_relevant(name):
    if not isinstance(name, str):
        return False
    keep = False
    # Mantén solo clases propias o main
    for pat in ["Manager::", "Worker::", "Reporter::", "main"]:
        if pat in name:
            keep = True
    return keep

# 3️⃣ Aplica filtro al DataFrame
df_logic_clean = df_logic[
    df_logic['callee_func'].apply(is_relevant) &
    df_logic['caller_func'].apply(is_relevant)
].copy()

# 4️⃣ Aplica pretty name
df_logic_clean['caller_func'] = df_logic_clean['caller_func'].apply(pretty_func)
df_logic_clean['callee_func'] = df_logic_clean['callee_func'].apply(pretty_func)

df_logic_clean[['thread_id','caller_func','callee_func']].head()

,thread_id,caller_func,callee_func
5,51361,main,Manager::Manager()
6,51361,Manager::Manager(),Worker::Worker()
87,51361,Manager::Manager(),Worker::Worker()
108,51361,Manager::Manager(),Worker::onJobDone(std::function<void (CalcJob ...
119,51361,Manager::onJobDone(),Manager::onJobDone()


In [14]:
df_logic = df_logic_clean

In [15]:
# --- Función robusta para extraer clase/namespace principal ---
def extract_participant(func):
    if not isinstance(func, str) or func == "??":
        return "Global"
    match = re.search(r'([a-zA-Z_][\w]*)::', func)
    if match:
        part = match.group(1)
    else:
        part = func.split("::")[0] if "::" in func else func
    if part.startswith("std") or " " in part:
        return None
    return part

# --- 1️⃣ Identificar participantes únicos ---
participants = set()

for func in pd.concat([df_logic['caller_func'], df_logic['callee_func']]):
    part = extract_participant(func)
    if part:
        participants.add(part)

participants = sorted(participants)
print("Participants:", participants)

Participants: ['Manager', 'Reporter', 'Worker', 'main']


In [23]:
# Normalizar timestamps al primero
t0 = df_logic['timestamp_ns'].min()
df_logic['timestamp_rel_ns'] = df_logic['timestamp_ns'] - t0

In [24]:
# --- 2️⃣ Generar código PlantUML ---
uml = ["@startuml"]

# Agregar participantes
for part in participants:
    uml.append(f"participant {part}")

# --- 3️⃣ Insertar llamadas con notas por hilo y timestamps ---
last_tid = None

for _, row in df_logic.sort_values('timestamp_rel_ns').iterrows():
    tid = row['thread_id']
    event = row['event']

    caller = extract_participant(row['caller_func']) or "Global"
    callee = extract_participant(row['callee_func']) or "Global"
    func = row['callee_func']

    # Timestamp en ms para legibilidad
    ts_ms = row['timestamp_rel_ns'] / 1e6

    # Nota de cambio de hilo
    if tid != last_tid:
        uml.append(f"note over {caller}: Thread {tid}")
        last_tid = tid

    # Flecha con timestamp
    if event == "ENTER":
        uml.append(f"{caller} -> {callee}: {func}\\n@ {ts_ms:.3f} ms")
    elif event == "EXIT":
        uml.append(f"{callee} --> {caller}: return")

uml.append("@enduml")

# --- 4️⃣ Mostrar en notebook listo para copy-paste ---
plantuml_code = "\n".join(uml)
display(Markdown(f"```plantuml\n{plantuml_code}\n```"))

```plantuml
@startuml
participant Manager
participant Reporter
participant Worker
participant main
note over main: Thread 51361
main -> Manager: Manager::Manager()\n@ 0.000 ms
Manager -> Worker: Worker::Worker()\n@ 0.005 ms
Worker --> Manager: return
Manager -> Worker: Worker::onJobDone(std::function<void (CalcJob const&)>)\n@ 0.457 ms
Manager -> Manager: Manager::onJobDone()\n@ 0.503 ms
Manager --> Manager: return
Worker --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 0.705 ms
Manager --> Manager: return
Manager --> main: return
main -> Manager: Manager::start()\n@ 0.743 ms
Manager -> Worker: Worker::start()\n@ 0.747 ms
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.764 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.776 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.789 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.801 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.814 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.903 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 0.915 ms
Worker --> Global: return
note over Global: Thread 51362
Global -> Worker: Worker::start()::{lambda()#1}& std::__get_helper<0ul, Worker::start()::{lambda()#1}>(std::_Tuple_impl<0ul, Worker::start()::{lambda()#1}>&)\n@ 1.311 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}&&>(std::remove_reference<Worker::start()::{lambda()#1}&&>::type&)\n@ 1.369 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 1.402 ms
Worker --> Global: return
Global -> Worker: Worker::start()::{lambda()#1}&& std::forward<Worker::start()::{lambda()#1}>(std::remove_reference<Worker::start()::{lambda()#1}>::type&)\n@ 1.426 ms
note over Manager: Thread 51361
Worker --> Manager: return
note over Global: Thread 51362
Worker --> Global: return
note over main: Thread 51361
Manager --> main: return
note over Global: Thread 51362
Global -> Worker: Worker::start()::{lambda()#1}::operator()() const\n@ 1.443 ms
note over main: Thread 51361
main -> Manager: Manager::submitJob(CalcJob const&)\n@ 1.447 ms
note over Worker: Thread 51362
Worker -> Worker: Worker::run()\n@ 1.451 ms
note over Manager: Thread 51361
Manager -> Worker: Worker::enqueueJob(CalcJob const&)\n@ 1.455 ms
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::submitJob(CalcJob const&)\n@ 1.700 ms
Manager -> Worker: Worker::enqueueJob(CalcJob const&)\n@ 1.704 ms
note over Worker: Thread 51362
Worker -> Worker: Worker::waitCondition()\n@ 1.792 ms
Worker --> Worker: return
Worker -> Worker: Worker::process(CalcJob&)\n@ 1.979 ms
Worker --> Worker: return
Manager -> Manager: Manager::onJobDone()\n@ 2.086 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 2.122 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 2.167 ms
Manager -> Reporter: Reporter::report(CalcJob const&)\n@ 2.176 ms
note over Manager: Thread 51361
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::submitJob(CalcJob const&)\n@ 2.240 ms
note over Reporter: Thread 51362
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)\n@ 2.242 ms
note over Manager: Thread 51361
Manager -> Worker: Worker::enqueueJob(CalcJob const&)\n@ 2.245 ms
note over Reporter: Thread 51362
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Manager: return
note over Manager: Thread 51361
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::submitJob(CalcJob const&)\n@ 2.522 ms
Manager -> Worker: Worker::enqueueJob(CalcJob const&)\n@ 2.526 ms
Worker --> Manager: return
Manager --> main: return
note over Worker: Thread 51362
Worker -> Worker: Worker::waitCondition()\n@ 2.824 ms
Worker --> Worker: return
Worker -> Worker: Worker::process(CalcJob&)\n@ 3.153 ms
Worker --> Worker: return
Manager -> Manager: Manager::onJobDone()\n@ 3.225 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 3.243 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 3.264 ms
Manager -> Reporter: Reporter::report(CalcJob const&)\n@ 3.268 ms
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)\n@ 3.285 ms
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Manager: return
Worker -> Worker: Worker::waitCondition()\n@ 3.400 ms
Worker --> Worker: return
Worker -> Worker: Worker::process(CalcJob&)\n@ 3.557 ms
Worker --> Worker: return
Manager -> Manager: Manager::onJobDone()\n@ 3.618 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 3.635 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 3.656 ms
Manager -> Reporter: Reporter::report(CalcJob const&)\n@ 3.660 ms
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)\n@ 3.666 ms
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Manager: return
Worker -> Worker: Worker::waitCondition()\n@ 3.765 ms
Worker --> Worker: return
Worker -> Worker: Worker::process(CalcJob&)\n@ 3.885 ms
Worker --> Worker: return
Manager -> Manager: Manager::onJobDone()\n@ 3.901 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 3.906 ms
Manager --> Manager: return
Manager -> Manager: Manager::onJobDone()\n@ 3.912 ms
Manager -> Reporter: Reporter::report(CalcJob const&)\n@ 3.913 ms
Reporter -> Reporter: Reporter::opToStr(CalcJob::Op)\n@ 3.914 ms
Reporter --> Reporter: return
Reporter --> Manager: return
Manager --> Manager: return
Worker -> Worker: Worker::waitCondition()\n@ 3.941 ms
Worker --> Worker: return
note over main: Thread 51361
main -> Manager: Manager::stop()\n@ 1003.251 ms
Manager -> Worker: Worker::stopWorker()\n@ 1003.261 ms
note over Worker: Thread 51362
Worker -> Worker: Worker::waitCondition()\n@ 1003.437 ms
Worker --> Worker: return
Worker --> Worker: return
Worker --> Global: return
note over Manager: Thread 51361
Worker --> Manager: return
Manager --> main: return
main -> Manager: Manager::~Manager()\n@ 1003.889 ms
Manager -> Worker: Worker::~Worker()\n@ 1003.894 ms
Manager -> Manager: Manager::onJobDone()\n@ 1003.920 ms
Manager --> Manager: return
Worker --> Manager: return
Manager --> main: return
@enduml
```

In [25]:
with open("../results/seq_diagram.puml","w") as fp:
    fp.write(plantuml_code)